# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns = None
pd.options.display.max_rows = None

### Загрузка данных

Загрузим данные и посмотрим основную информацию.

In [2]:
df0 = pd.read_csv("/datasets/geo_data_0.csv")
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [3]:
df0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [4]:
df1 = pd.read_csv("/datasets/geo_data_1.csv")
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [6]:
df2 = pd.read_csv("/datasets/geo_data_2.csv")
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
df2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


### Пропуски

In [8]:
df0.isnull().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [9]:
df1.isnull().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [10]:
df2.isnull().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Пропусков нет. С данными можно работать далее.

### Изучим зависимость данных

Посмотрим корреляцию признаков в регионах

In [11]:
df0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [12]:
df1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [13]:
df2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Довольно сильно коррелирует признак f2 и целевой признак для второго региона. Но признаки между собой не сильно зависимы, поэтому оставляем все три. 

## Обучение и проверка модели

### Разбивка данных на обучающую и валидационную выборки 

In [14]:
target0 = df0['product']
features0 = df0.drop(['id','product'], axis=1)
features0_train, features0_valid, target0_train, target0_valid = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)

target1 = df1['product']
features1 = df1.drop(['id','product'], axis=1)
features1_train, features1_valid, target1_train, target1_valid = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)

target2 = df2['product']
features2 = df2.drop(['id','product'], axis=1)
features2_train, features2_valid, target2_train, target2_valid = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)

### Обучение модели

In [15]:
model0 = LinearRegression()
model0.fit(features0_train,target0_train)

model1 = LinearRegression()
model1.fit(features1_train,target1_train)

model2 = LinearRegression()
model2.fit(features2_train,target2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Предсказания на валидационной выборке

In [16]:
predict0 = model0.predict(features0_valid)
predict1 = model1.predict(features1_valid)
predict2 = model2.predict(features2_valid)

### Средний запас предсказанного сырья и RMSE модели

In [17]:
def metrics(target,predict,n):
    mse = mean_squared_error(target,predict)
    rmse = round(mse**0.5,4)
    mean = round(predict.mean(),4)
    display(f'Регион {n}. Средний запас предсказанного сырья:{mean}  и RMSE модели: {rmse}')

In [18]:
metrics(target0_valid,predict0,1)

'Регион 1. Средний запас предсказанного сырья:92.5926  и RMSE модели: 37.5794'

In [19]:
metrics(target1_valid,predict1,2)

'Регион 2. Средний запас предсказанного сырья:68.7285  и RMSE модели: 0.8931'

In [20]:
metrics(target2_valid,predict2,3)

'Регион 3. Средний запас предсказанного сырья:94.965  и RMSE модели: 40.0297'

Средний запас предсказанного сырья для первого и третьего региона близок, как из RMSE модели. <br>
При этом для второй средний запас меньше и rmse меньше 1. Возможно это связано с тем, что для вторго региона корреялция меджду признаком f2 и целевым принаком близка к 1, и предсказания точнее.

## Подготовка к расчёту прибыли

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [21]:
BUDGET = 10000000000
AMOUNT_ALL = 500
AMOUNT_B = 200
BARREL1000_PRICE = 450000

BUDGET_WELL = BUDGET/AMOUNT_B
VOLUME = round(BUDGET_WELL/BARREL1000_PRICE,4)

display(f'Объем сырья для безубыточной разработки новой скважины: {VOLUME}')

'Объем сырья для безубыточной разработки новой скважины: 111.1111'

Рассчитаем средний запас в каждом регионе.

In [22]:
mean_0 = df0['product'].mean()
mean_1 = df1['product'].mean()
mean_2 = df2['product'].mean()

display(f'Средний запас в регионе 0: {mean_0}')
display(f'Средний запас в регионе 1: {mean_1}')
display(f'Средний запас в регионе 2: {mean_2}')

'Средний запас в регионе 0: 92.50000000000001'

'Средний запас в регионе 1: 68.82500000000002'

'Средний запас в регионе 2: 95.00000000000004'

Средний запас в каждом регионе ниже объема для безубыточной разработки. Но возможно данных много и есть скажины пустые или с довольно низким объемом.

## Расчёт прибыли и рисков 

### Расчет прибыли по выбранным скважинам и предсказаниям модели

In [23]:
def profit (predictions,target):
    predict_sort = pd.Series(predictions, index=target.index)
    predict_sort = predict_sort.sort_values(ascending=False)
    target_sort = target[predict_sort.index][:AMOUNT_B]
    sum_sort = target_sort.sum()
    return (sum_sort * BARREL1000_PRICE - BUDGET)
 
income0 = profit(predict0,target0_valid)
print ("Прибыль для полученного объёма сырья выборки df0:", (income0/1000000000).round(2),"млрд.руб.")

Прибыль для полученного объёма сырья выборки df0: 3.32 млрд.руб.


In [24]:
income1 = profit(predict1,target1_valid)
print ("Прибыль для полученного объёма сырья выборки df1:", (income1/1000000000).round(2),"млрд.руб.")

Прибыль для полученного объёма сырья выборки df1: 2.42 млрд.руб.


In [25]:
income2 = profit(predict2,target2_valid)
print ("Прибыль для полученного объёма сырья выборки df2:", (income2/1000000000).round(2),"млрд.руб.")

Прибыль для полученного объёма сырья выборки df2: 2.71 млрд.руб.


###  Риски и прибыль для каждого региона

In [26]:
def bootstrap(predictions, target):
    state = np.random.RandomState(0)
    values = []
 
    for i in range(1000):
        target_subsamples = target.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        predictions_subsamples = predictions[target_subsamples.index]
        values.append(profit(predictions_subsamples, target_subsamples))
 
    values = pd.Series(values)
    lower = round(values.quantile(0.025)/1000000,2)
    upper = round(values.quantile(0.975)/1000000,2)
    mean_profit = round((values.mean()/1000000),2)
    loss = (np.mean(values < 0)*100)
 
    return mean_profit, loss, lower, upper

In [27]:
mean0_profit, loss0, lower0, upper0 = bootstrap(predict0, target0_valid)
mean1_profit, loss1, lower1, upper1 = bootstrap(predict1, target1_valid)
mean2_profit, loss2, lower2, upper2 = bootstrap(predict2, target2_valid)

In [28]:
display(f'Для 0 региона - Средняя прибыль:{mean0_profit} млн., Доверительный интревал: {lower0}-{upper0} млн., Риск убытков:{loss0} %')
display(f'Для 1 региона - Средняя прибыль:{mean1_profit} млн., Доверительный интревал: {lower1}-{upper1} млн., Риск убытков:{loss1} %')
display(f'Для 2 региона - Средняя прибыль:{mean2_profit} млн., Доверительный интревал: {lower2}-{upper2} млн., Риск убытков:{loss2} %')

'Для 0 региона - Средняя прибыль:410.3 млн., Доверительный интревал: -163.03-972.21 млн., Риск убытков:7.7 %'

'Для 1 региона - Средняя прибыль:507.55 млн., Доверительный интревал: 87.32-915.14 млн., Риск убытков:0.6 %'

'Для 2 региона - Средняя прибыль:412.26 млн., Доверительный интревал: -131.31-947.92 млн., Риск убытков:7.5 %'

Предпочтительным для разработки скажин является регион с номером 1(2ой). Так как риск убытков всего 0,6%. А так же средняя прибыль выше чем в других регионах.